In [110]:



import cv2
import mediapipe as mp
import csv
from datetime import datetime, timedelta
import math
import numpy as np

#CSV 파일에 헤더를 추가

with open('710.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['R','G','B', 'Openness', 'Width', 'Label'])


# Mediapipe의 Face Mesh 모델 로드
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 웹캠에서 영상을 받아오는 VideoCapture 객체 생성
cap = cv2.VideoCapture(0)

start_time = datetime.now()
elapsed_time = timedelta(seconds=0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 캡처 실패, 종료합니다.")
        break

    frame = cv2.flip(frame, 1)
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 얼굴 검출 및 랜드마크 추출
    results = face_mesh.process(rgb_frame)

    
    if results.multi_face_landmarks:
     
        for face_landmarks in results.multi_face_landmarks:

            mouth_area=frame.copy()

            landmark_indices =[0,61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291,37,39,40,267,269,270]
            
            landmarks = [face_landmarks.landmark[i] for i in landmark_indices]

            landmark_indices2 =[12,13,14,15,16,17,18]
            

            landmarks = [face_landmarks.landmark[i] for i in landmark_indices]
            landmarks2 = [face_landmarks.landmark[i] for i in landmark_indices2]
            
            h, w, _ = frame.shape
            x_1 = int(np.mean([landmark.x * w for landmark in landmarks]))
            y_1 = int(np.mean([landmark.y * h for landmark in landmarks]))-5

        
            cv2.circle(frame, (x_1, y_1),2, (0, 255, 0), -1)



            # 각각의 RGB 값 구하고 평균
            
            rgb_value = rgb_frame[y_1, x_1]
            r,g,b = rgb_value[0], rgb_value[1], rgb_value[2]


                    

                #웹캠과의 거리에 따라 입의 가로길이와 세로길이가 변경함에 따라 기준값으로 얼굴너비를 설정

            face_width = math.sqrt(
            (face_landmarks.landmark[172].x - face_landmarks.landmark[264].x) ** 2 +
            (face_landmarks.landmark[172].y - face_landmarks.landmark[264].y) ** 2
        )
            face_width=20*face_width
        


            #입의 세로길이=>고중저모음 판단기준

            upper_lip_bottom = (face_landmarks.landmark[12].x, face_landmarks.landmark[12].y)
            lower_lip_top = (face_landmarks.landmark[14].x, face_landmarks.landmark[14].y)
            mouth_openness = (lower_lip_top[1] - upper_lip_bottom[1])/face_width
            mouth_openness=round(mouth_openness,4)


        
            # 입의 가로길이를 계산=>원순/평순 판단기준
            
            lip_left =(face_landmarks.landmark[61].x, face_landmarks.landmark[61].y)
            lip_right=(face_landmarks.landmark[291].x, face_landmarks.landmark[291].y)
            mouth_width=(lip_right[0]-lip_left[0])/face_width
            mouth_width=round(mouth_width,4)


            
            # 경과 시간이 0.5초 이상일 때만 CSV 파일에 추가
            if elapsed_time.total_seconds() >= 0.5:
                # 시간 단위로 입안 명암도, 입 벌림 정도, 입의 가로길이를 CSV 파일에 추가
                with open('710.csv', mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([r,g,b,mouth_openness,mouth_width,7]) #label변경사항
                
                # 경과 시간 초기화
                elapsed_time = timedelta(seconds=0)
            else:
                # 경과 시간 누적
                elapsed_time += datetime.now() - start_time
                start_time = datetime.now()
            

            # 입 벌림 정도, 입안 명암도, 입의 가로길이를 프레임에 표시
           
            cv2.putText(frame, f"Mouth Openness: {mouth_openness:.4f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"Mouth Width: {mouth_width:.4f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame, f"RGB: ({r:},{g:},{b:})", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
           
            
    # 영상 출력
    cv2.imshow('Face Mesh', frame)
    
    # 종료 키 (q) 입력 시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 


# 자원 해제
cap.release()
cv2.destroyAllWindows()


In [ ]:
# 새로운 데이터에 대한 예측 결과
predictions = model.predict(new_features)

# 예측된 클래스 저장할 리스트
predicted_classes = []

# 각 데이터에 대한 예측 클래스 선택
for prediction in predictions:
    predicted_class = np.argmax(prediction)  # 가장 높은 확률을 가지는 클래스 선택
    predicted_classes.append(predicted_class)

# 5개씩 묶어서 최빈값 계산 및 출력
num_groups = len(predicted_classes) // 10
for i in range(num_groups):
    group = predicted_classes[i*10:(i+1)*10]
    most_common_class = max(set(group), key=group.count)  # 최빈값 계산
    print(f"{i+1}번째 묶음에서 가장 많이 나온 클래스: {most_common_class}")
